In [1]:
from flask import Flask, render_template
import pandas as pd
import numpy as np
from queue import PriorityQueue

In [2]:
manifest_name = "SS TEST SHIP"
employee_name = "John Doe"
shipCase = 2

nrows=8
ncols=12

# manifest = []
# for case in range(1,6):
#     manifest.append( pd.read_csv('./ship_cases/ShipCase'+str(case)+'.txt',header=None,names=["col","row","weight","name"]) )
#     manifest[case-1]['col'] = manifest[case-1]['col'].str.replace("[","", regex=True)
#     manifest[case-1]['row'] = manifest[case-1]['row'].str.replace("]","", regex=True)
#     manifest[case-1]['weight'] = manifest[case-1]['weight'].str.replace("{","", regex=True).replace("}","", regex=True)

In [3]:
testManifest = ( pd.read_csv('./ship_cases/ShipCase'+str(shipCase)+'.txt',header=None,names=["col","row","weight","cont"]) )
testManifest['col'] = testManifest['col'].str.replace("\[| ","", regex=True)
testManifest['row'] = testManifest['row'].str.replace("\]| ","", regex=True)
testManifest['weight'] = testManifest['weight'].str.replace("{|}| ","", regex=True)
testManifest['cont'] = (testManifest['cont'].str[1:])

testManifest.head()

,col,row,weight,cont
0,01,01,00000,NAN
1,01,02,00000,NAN
2,01,03,00000,NAN
3,01,04,00120,Ram
4,01,05,00000,UNUSED


In [4]:
class Cell:
    def __init__(self, col, row, weight, cont):
        self.col = col
        self.row = row
        self.weight = weight
        seld.cont = cont
#not actually using this

In [5]:
#creating 2d np array for ship
testManifestArray = np.array(testManifest.loc[:]).reshape(nrows,ncols,4)
print(testManifestArray[0,1])

['01' '02' '00000' 'NAN']


In [6]:
#creating 2d np array for buffer
testBuffer = np.zeros(4*24).reshape(4,24)
print(testBuffer)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


####

In [7]:
def printState(array):
    for j in range(7,-1,-1):
        print("")
        for i in array[j]:
            if(i[3]=="UNUSED"):
                print("[  ]", end="\t")
            else:
                print(i[3], end="\t")
#printState(testManifestArray)

In [8]:
printState(testManifestArray)


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	[  ]	Owl	NAN	NAN	NAN	

### Defining a State

In [9]:
#Need Matrix of Ship Containers
#Need Matrix of Buffer Zone
#Need Col # the Crane is picking from
class State:
    def __init__(self, ship, topLayer=[], buffer=testBuffer):
        self.ship = ship
        self.buffer = buffer
        self.topLayer = topLayer

    def __str__(self):
        printState(self.ship)
        #return f"\n\nPick Col: {self.colPos}"
        return ""

In [10]:
#decorator clas for use with branching
class StateWrapper:
    def __init__(self, state, colPick=-1, h=20, moves = 0, cost=0, movesList = []):
        self.state = state
        self.colPick = colPick
        self.h = h
        self.moves = moves
        self.cost = cost
            #of the form [(x1,y1), (x2,y2)]
        self.movesList = movesList

    def __str__(self):
        printState(self.state.ship)
        t=""
        if self.colPick>0:
            t=f"Pick Col: {self.colPick}"
        return t+f"\n\nh: {self.h}\n# of Moves: {self.moves}\nCost: {self.cost}\nMovesList: {self.movesList}"

In [11]:
print(StateWrapper(State(testManifestArray), 1))


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	[  ]	Owl	NAN	NAN	NAN	Pick Col: 1

h: 20
# of Moves: 0
Cost: 0
MovesList: []


### Branching Factor is 12 (number of col)

In [12]:
#To branch we place our picked container (top most of selected col) on top of highest container of another col


#### Helper Function to find Highest Container in Stack

In [13]:
def stackHeight(array, col, nrows=nrows):
    for row in range(nrows-1,-1,-1):
        if(array[row,col,3] != "UNUSED"):
            return row
    return(-1)
    #-1 means no containers/blocked spaces in col

In [14]:
testCol=4
print(testManifestArray[:,testCol])
stackHeight(testManifestArray,testCol)

[['01' '05' '00000' 'UNUSED']
 ['02' '05' '00000' 'UNUSED']
 ['03' '05' '00000' 'UNUSED']
 ['04' '05' '00000' 'UNUSED']
 ['05' '05' '00000' 'UNUSED']
 ['06' '05' '00000' 'UNUSED']
 ['07' '05' '00000' 'UNUSED']
 ['08' '05' '00000' 'UNUSED']]


-1

In [15]:
def heights(array, cols=ncols):
    h = []
    for col in range(0,cols):
        h.append(stackHeight(array,col) + 1)
    return h

heights(testManifestArray)

[3, 2, 1, 1, 0, 0, 0, 0, 1, 1, 1, 2]

#### Helper function to find top layer

In [16]:
def updateTop(array, cols=ncols):
    top = []
    for i in range(0,cols):
        h = stackHeight(array,i)
        if(h!=-1):
            top.append( np.append(array[h,i,2:4],h) )
        else:
            top.append(np.array(['-99999','NONAME',h]))
    return np.array(top)

tmp = StateWrapper(State(testManifestArray, topLayer=updateTop(testManifestArray)), 1)
print(tmp.state.topLayer)

[['00040' 'Cat' 2]
 ['00050' 'Dog' 1]
 ['00000' 'NAN' 0]
 ['00120' 'Ram' 0]
 ['-99999' 'NONAME' '-1']
 ['-99999' 'NONAME' '-1']
 ['-99999' 'NONAME' '-1']
 ['-99999' 'NONAME' '-1']
 ['00035' 'Owl' 0]
 ['00000' 'NAN' 0]
 ['00000' 'NAN' 0]
 ['00000' 'NAN' 1]]


#### Helper to find if can pick container

In [17]:
def pickable(array, col, nrows=nrows):
    h = stackHeight(array,col,nrows)
    if(array[h,col,3]=="NAN" or h==-1):
        return False
    return True

#### Place container to new col

In [51]:
def moveContainer(array, col, newCol):
    if(col==newCol):
        return -1
    #find where to place container
    h1 = stackHeight(array,col)
    h2 = stackHeight(array,newCol) +1 
        
    newArray = array.copy()
    newArray[h2,newCol,2:4] = newArray[h1,col,2:4]  
    newArray[h1,col,2:4] = ['00000','UNUSED']
    
#     print((newArray[h1]))
    return newArray

print(State(testManifestArray))
print()
s1 = State(moveContainer(testManifestArray,3,6))
s2 = State(moveContainer((moveContainer(testManifestArray,3,6)),8,5))
print(s1)
print(s2)



[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	[  ]	Owl	NAN	NAN	NAN	


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	[  ]	[  ]	[  ]	Ram	[  ]	Owl	NAN	NAN	NAN	

[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]

#### Cost of Moving from col x to col y

In [77]:
def costCol(array, curCol, newCol, empty=0):
    hs = heights(array)
    maxHeight = max(hs)
    
    h1 = stackHeight(array,col=curCol) + 1
    h2 = stackHeight(array,col=newCol) + 1
    
    
    if(curCol<newCol):
        localMax = max(hs[curCol+1:newCol]) 
        print(localMax,h1,h2)
        if(h2>=h1 and localMax<h1):
            return abs(0) + abs(curCol-newCol) + abs(localMax-h2-1) - empty
        if(h2>=h1 or localMax>=h1):
            return abs(localMax-h1) + abs(newCol-curCol) + abs(localMax-h2-1) 
        else:
            return abs(h1-localMax) + abs(newCol-curCol) + abs(localMax-h2-1) 
    else:
        localMax = max(hs[newCol+1:curCol]) + 1
#         print(localMax,h1,h2)
        if(h2>=h1 and localMax<h1):
            return abs(localMax-h1) + abs(curCol-newCol) + abs(localMax-h2-1) - empty
        if(h2>=h1 or localMax>=h1):
            return abs(localMax-h1) + abs(curCol-newCol) + abs(localMax-h2-1)
        else:
            return abs(h1-localMax) + abs(curCol-newCol) + abs(localMax-h2-1)   
        
print(costCol(s1.ship , 6, 8))

0 1 1
4


In [80]:
def costCol(array, curCol, newCol, empty=0):
    hs = heights(array)
    maxHeight = max(hs)
    
    h1 = stackHeight(array,col=curCol) + 1
    h2 = stackHeight(array,col=newCol) + empty
    
    
    if(curCol<newCol):
        localMax = max(hs[curCol+1:newCol+1]) + 1
#         print(localMax,h1,h2)
        if(h2>=h1 or localMax>=h1):
            return abs(localMax-h1) + abs(newCol-curCol) + abs(localMax-h2-1)
        else:
            return abs(h1-localMax) + abs(newCol-curCol) + abs(localMax-h2-1) 
    else:
        localMax = max(hs[newCol+1:curCol+1]) + 1
#         print(localMax,h1,h2)
        if(h2>=h1 or localMax>=h1):
            return abs(localMax-h1) + abs(curCol-newCol) + abs(localMax-h2-1)  
        else:
            return abs(h1-localMax) + abs(curCol-newCol) + abs(localMax-h2-1)   
        
print(costCol(s1.ship , 6, 8))

4


### Priority Queue Helper

In [56]:
#Code Chunk from official python priority queue page
from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)

## Load Unload Problem

In [57]:
unload = []
load = []

#### Load Function

In [58]:
#

#### Unload Function

### Load/Unload Loop

## Balancing Problem

In [59]:
#Function for testing balance
def balanceScore(array, cols=ncols):
    split=cols/2
    left = sum(((array[:,:int(split),2]).flatten()).astype(int))
    #print(left)
    right = sum(((array[:,int(split):,2]).flatten()).astype(int))
    #print(right)
    
    if(left==right):
        return 1
    if(left==0 or right==0):
        return 0
    
    return min(left,right)/max(left,right)
#balanceScore(testManifestArray)

In [60]:
#Function for accepting balance
#default threshold of 10% 
def balanced(state, cols=ncols, thresh=.1):
    array = state.state.ship
    if(balanceScore(array, cols) > (1-thresh)):
        return True
    return False
#balanced(testManifestArray)

### Heuristic (Balance)

In [61]:
def totalWeight(array):
    return sum(((array[:,:,2]).flatten()).astype(int))

def leftWeight(array, cols=ncols):
    split=cols/2
    return sum(((array[:,:int(split),2]).flatten()).astype(int))

def rightWeight(array, cols=ncols):
    split=cols/2
    return sum(((array[:,int(split):,2]).flatten()).astype(int))

def removeEmpty(array):
    filterArray = ((array[:,:,3] != 'UNUSED') == (array[:,:,3] != 'NAN'))
    return (array[filterArray])

def heavySide(state, cols=ncols):
    array = state.state.ship
    left = leftWeight(array,cols)
    right = rightWeight(array,cols)
    if(left < right):
        return 1
    return 0

def balanceHeuristicDroped(state, cols=ncols):
    #Given state, how balanced is it and what potential does it have?
    
    array = state.state.ship
    
    balanceMass = totalWeight(array)/2
    split = cols/2
    
    #which side is heavier
    side = heavySide(state)
        
    left = leftWeight(array,cols)
    right = rightWeight(array,cols)
    
    deficit = abs(balanceMass - max(left, right))

    if(side==0):
        weights = removeEmpty((array[:,:int(split)]))
    else:
        weights = removeEmpty((array[:,int(split):]))
    
    #list of cells sorted by weight
    sortedList = sorted(weights, key=lambda x: x[2], reverse=True)
    
    newDef = deficit
    h = 0
    for i in sortedList:
        if (newDef - int(i[2]) > 0):
            newDef -= int(i[2])
            h+=1
            
    #h is amount of containers needed to move to get close to balance
    return h, sortedList


def balanceHeuristicPicked(state, colPick=-1, cols=ncols):
    #Picking from col pickCol, how good is this state?
    #what is the dist needed to drop into col on other side
    
    if(state.colPick >= 0):
        colPick = state.colPick
    array = state.state.ship
    
    side = heavySide(state)
    
    heightsArray = np.array(heights(array))
    
    #check if picking from heavy side    
    if(side==0 and colPick<cols/2):
        if( (heightsArray<nrows-1).all() ):
#             print("side 0, moving to",cols/2)
            return cols/2 - colPick

        #there are full cols
        print("left")
        for i in range(cols/2, cols):
            if(heightsArray[i] < nrows-1):
                return i - colPick
    
    if(side==1 and colPick>=cols/2):
        if( (heightsArray<nrows-1).all() ):
            return colPick - ((cols/2)-1)
        
        #there are full cols
        print("right")
        for i in range((cols/2)-1, -1, -1):
            if(heightsArray[i] < nrows-1):
                return colPick-i
            
    #pulling from side that is lighter
    
    #h is slightly worse than worst case moving from heavier side
    return 1 + cols/2

wrapTest = StateWrapper(State(testManifestArray), colPick= 4)
# wrapTest.movesList.append([[currentState.state.ship[stackHeight(currentState.state.ship,col=currentState.colPick), currentState.colPick], currentState.colPick],[stackHeight(newDropState.state.ship,col=newCol),newCol]])

print(wrapTest)


h, sort = balanceHeuristicDroped(wrapTest)
balanceHeuristicPicked(wrapTest)



[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	[  ]	Owl	NAN	NAN	NAN	Pick Col: 4

h: 20
# of Moves: 0
Cost: 0
MovesList: []


2.0

### Branching Loop

In [62]:
#when tracking moves, compensate coordinates +1 for dropoff location to match pick up
    #**coordinates start at 1, not 0**
comp = 1

#initial state:
initialState = StateWrapper(State(testManifestArray))

#duplicate states checker
duplicateState = []

#legal final states
finalState = []


#Create PriorityQueue for queing states
queueP = PriorityQueue()
queueD = PriorityQueue()

#queue.put((0,initialState))

#Choose which container to move
    #creating and adding those states
    

queueD.put(PrioritizedItem(1, initialState))
            
depth = 0

#Choose where to move the container
    #creating and adding those states
while True:
    
    if not queueD.empty():
        currentState=queueD.get().item
        if(balanced(currentState)):
            print("Already Balanced")
            finalState.append(currentState)
            break;
    #Choose which container to move
        #creating and adding those states
        for w in range(0,ncols):
            dropped=-1
            if len(currentState.movesList) > 0:
                    #cost of moving crane from dropped to new pickup
                    dropped = currentState.movesList[-1][1][1]-1
            
            if(w!=dropped):
                extraCost=0
                if len(currentState.movesList) > 0:
                    extraCost = costCol(currentState.state.ship,currentState.movesList[-1][1][1]-1,w,empty=1)
                    print("extra",extraCost)

                newPickState = StateWrapper(currentState.state, colPick=w, h=balanceHeuristicPicked(currentState, colPick=w), moves=currentState.moves, cost=currentState.cost + extraCost, movesList=currentState.movesList.copy())

                if(pickable(newPickState.state.ship, w)):
                    queueP.put(PrioritizedItem(newPickState.h,newPickState))
                
    
    while not queueP.empty():
        currentState = queueP.get().item
        print("\n\nPick State", currentState.state.ship[stackHeight(currentState.state.ship,col=currentState.colPick), currentState.colPick], "with h =",currentState.h)
        if(balanced(currentState)):
            print("Already Balanced")
            finalState.append(currentState)
            break;
        #Create new branch
        for newCol in range(0,ncols):
            #try to place container in each col
            if(stackHeight(currentState.state.ship,currentState.colPick)<nrows and newCol!=currentState.colPick):
                #does not allow if overflow or where it already was
                dropH, sortedList = balanceHeuristicDroped(state=currentState)
                newDropState = StateWrapper(State(moveContainer(currentState.state.ship,currentState.colPick,newCol)), h=dropH, moves=currentState.moves+1, cost=currentState.cost + costCol(currentState.state.ship,curCol=currentState.colPick,newCol=newCol), movesList=currentState.movesList.copy())
                
                newDropState.movesList.append([[currentState.state.ship[stackHeight(currentState.state.ship,col=currentState.colPick), currentState.colPick].tolist()],[stackHeight(newDropState.state.ship,col=newCol)+comp,newCol+comp]])
                
                if(balanced(newDropState)):
                    print("[BALANCED]Droping container at", newCol, "for h =", newDropState.h)
                    finalState.append(newDropState)
#                     break;
                elif((newDropState not in duplicateState)):
                    duplicateState.append(newDropState.state)
                #add h of currentState (parent) and new Drop
                    queueD.put(PrioritizedItem(currentState.h + newDropState.h,newDropState))
                    print("\t  Droping container at", newCol, "for h =", newDropState.h)
    depth+=1

    if (depth>=20 or len(finalState)>3):
        break



Pick State ['01' '04' '00120' 'Ram'] with h = 3.0
	  Droping container at 0 for h = 1
	  Droping container at 1 for h = 1
	  Droping container at 2 for h = 1
	  Droping container at 4 for h = 1
	  Droping container at 5 for h = 1
	  Droping container at 6 for h = 1
	  Droping container at 7 for h = 1
	  Droping container at 8 for h = 1
	  Droping container at 9 for h = 1
	  Droping container at 10 for h = 1
	  Droping container at 11 for h = 1


Pick State ['02' '02' '00050' 'Dog'] with h = 5.0
	  Droping container at 0 for h = 1
	  Droping container at 2 for h = 1
	  Droping container at 3 for h = 1
	  Droping container at 4 for h = 1
	  Droping container at 5 for h = 1
	  Droping container at 6 for h = 1
	  Droping container at 7 for h = 1
	  Droping container at 8 for h = 1
	  Droping container at 9 for h = 1
	  Droping container at 10 for h = 1
	  Droping container at 11 for h = 1


Pick State ['03' '01' '00040' 'Cat'] with h = 6.0
	  Droping container at 1 for h = 1
	  Droping c

In [63]:
# for x in duplicateState:
#       print(x,"\n")

In [64]:
for x in finalState:
    print(x,"\n")


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Owl	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	[  ]	[  ]	[  ]	[  ]	Ram	[  ]	NAN	NAN	NAN	

h: 0
# of Moves: 2
Cost: 17
MovesList: [[[['01', '04', '00120', 'Ram']], [1, 8]], [[['01', '09', '00035', 'Owl']], [4, 1]]] 


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	Owl	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  

In [65]:
print("[initial]",initialState,"\n\n\t\t\t\t||\n\t\t\t\t\/\n")
best = sorted(finalState,key=lambda x: x.cost)[0]
print(best)
print("Best state's balance",balanceScore(best.state.ship))

[initial] 
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	Ram	[  ]	[  ]	[  ]	[  ]	Owl	NAN	NAN	NAN	

h: 20
# of Moves: 0
Cost: 0
MovesList: [] 

				||
				\/


[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
Cat	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	
NAN	Dog	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	[  ]	NAN	
NAN	NAN	NAN	[  ]	[  ]	Owl	[  

In [66]:
best.movesList[-1][1][1]

6